# 5.2
## TensorFlow スタイルによるプログラミング

In [1]:
# 5-02-12 CIFAR-10 データセットの読み込み
# p.299

from tensorflow import keras
from tensorflow.keras import datasets

(x_train, t_train), (x_test, t_test) = datasets.cifar10.load_data()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [2]:
# 5-02-13 CNNモデルの定義
# p.299

from tensorflow.keras import layers

class CNN(keras.Model):
    def __init__(self, output_dim):
        super().__init__()
        weight_decay = 1e-4
        self.c1 = layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            padding='same',
            input_shape=x_train[0].shape,
            kernel_regularizer=keras.regularizers.l2(weight_decay), 
            activation='relu'
            )
        self.p1 = layers.MaxPooling2D(pool_size=(2, 2))
        self.c2 = layers.Conv2D(
            filters=128,
            kernel_size=(3, 3),
            padding='same',
            kernel_regularizer=keras.regularizers.l2(weight_decay), 
            activation='relu'
            )
        self.p2 = layers.MaxPooling2D(pool_size=(2, 2))
        self.c3 = layers.Conv2D(
            filters=256,
            kernel_size=(3, 3),
            padding='same',
            kernel_regularizer=keras.regularizers.l2(weight_decay), 
            activation='relu'
            )
        self.p3 = layers.MaxPooling2D(pool_size=(2, 2))
        self.f1 = layers.Flatten()
        self.d1 = layers.Dropout(0.4)
        self.l1 = layers.Dense(512, activation='relu')
        self.l2 = layers.Dense(10, activation='softmax')
        self.ls = [ self.c1, self.p1, self.c2, self.p2, self.c3, 
                  self.p3, self.f1, self.d1, self.l1, self.l2 ]
        
    def call(self, x):
        for layer in self.ls:
            x = layer(x)
        return x

In [3]:
# 5-02-14 損失関数の定義
# p.303

cce = keras.losses.SparseCategoricalCrossentropy()

def loss(t, y):
    return cce(t, y)

In [4]:
# 5-02-15 オプティマイザー、損失と精度の記録
# p.303

optimizer = keras.optimizers.SGD(learning_rate=0.1)

train_loss = keras.metrics.Mean()
train_acc = keras.metrics.SparseCategoricalAccuracy()
val_loss = keras.metrics.Mean()
val_acc = keras.metrics.SparseCategoricalAccuracy()

In [5]:
# 5-02-16 パラメータの更新処理
# p.304

import tensorflow as tf

@tf.function
def train_step(x, t):
    with tf.GradientTape() as tape:
        outputs = model(x)
        tmp_loss = loss(t, outputs)
    grads = tape.gradient(tmp_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(tmp_loss)
    train_acc(t, outputs)

In [6]:
# 5-02-17 検証データでモデルを評価する
# p.305

def val_step(x, t):
    preds = model(x)
    tmp_loss = loss(t, preds)
    val_loss(tmp_loss)
    val_acc(t, preds)

### 学習の実行

In [7]:
# 5-02-18 訓練データを拡張処理して学習を行う
# p.308
# [自分へのメモ] %%time で実行時間を測るのは別のcellで行う

from sklearn.utils import shuffle

epochs = 100
batch_size = 64
train_steps = len(x_train) * 0.8 / batch_size
val_steps = len(x_train) * 0.2 / batch_size

model = CNN(10)

datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

training_generator = datagen.flow(
    x_train, 
    t_train,
    batch_size=batch_size,
    subset='training'
    )

validation_generator = datagen.flow(
    x_train,
    t_train,
    batch_size=batch_size,
    subset='validation'
    )

In [8]:
%%time

for epoch in range(epochs):
    step_counter = 0
    for x_batch, t_batch in training_generator:
        train_step(x_batch, t_batch)
        step_counter += 1
        if step_counter >= train_steps:
            break
    v_step_counter = 0
    for x_val_batch, t_val_batch in validation_generator:
        val_step(x_val_batch, t_val_batch)
        v_step_counter += 1
        if v_step_counter >= val_steps:
            break
    # 1 epoch 毎の結果を出力する
    print(f'epoch: {epoch+1}  '
          f'train_loss: {train_loss.result():.4f}  '
          f'train_acc: {train_acc.result():.4f}  '
          f'val_loss: {val_loss.result():.4f}  '
          f'val_acc: {val_acc.result():.4f}'
         )

epoch: 1  train_loss: 1.9456  train_acc: 0.2958  val_loss: 1.7882  val_acc: 0.3468
epoch: 2  train_loss: 1.7338  train_acc: 0.3746  val_loss: 1.5983  val_acc: 0.4252
epoch: 3  train_loss: 1.6072  train_acc: 0.4207  val_loss: 1.4879  val_acc: 0.4649
epoch: 4  train_loss: 1.5137  train_acc: 0.4557  val_loss: 1.4105  val_acc: 0.4956
epoch: 5  train_loss: 1.4376  train_acc: 0.4848  val_loss: 1.3438  val_acc: 0.5206
epoch: 6  train_loss: 1.3729  train_acc: 0.5092  val_loss: 1.2910  val_acc: 0.5395
epoch: 7  train_loss: 1.3171  train_acc: 0.5303  val_loss: 1.2420  val_acc: 0.5572
epoch: 8  train_loss: 1.2674  train_acc: 0.5488  val_loss: 1.2070  val_acc: 0.5696
epoch: 9  train_loss: 1.2232  train_acc: 0.5652  val_loss: 1.1800  val_acc: 0.5807
epoch: 10  train_loss: 1.1838  train_acc: 0.5796  val_loss: 1.1463  val_acc: 0.5931
epoch: 11  train_loss: 1.1478  train_acc: 0.5928  val_loss: 1.1160  val_acc: 0.6041
epoch: 12  train_loss: 1.1152  train_acc: 0.6047  val_loss: 1.0889  val_acc: 0.6140
e

In [9]:
# 5-02-19 テストデータで学習済みモデルを評価する
# p.310

test_loss = keras.metrics.Mean()
test_acc = keras.metrics.SparseCategoricalAccuracy()

def test_step(x, t):
    preds = model(x)
    tmp_loss = loss(t, preds)
    test_loss(tmp_loss)
    test_acc(t, preds)

x_test = x_test.astype('float32') / 255
test_step(x_test, t_test)

print(f'test_loss: {test_loss.result():.4f}  test_acc: {test_acc.result():.4f}')

test_loss: 0.8711  test_acc: 0.8203
